# Projeto 1 - Ciência dos Dados

Nome: Andressa Silva de Oliveira

Nome: Luana Mitsudo Coelho

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'xiaomi.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo xiaomi.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Valor
0,@miuirom miui 12 no rn8 xiaomi cade?,1
1,rt @dani13_f: meu cell note 8 xiaomi já tá com...,1
2,@notfakeholmes @brenomarquesof @arcanjobeni 3 ...,1
3,"@juliaaacras o da xiaomi é excelente, sem mais",1
4,@rrlleal00 xiaomi mi 9 se. comprei na amazon e...,0


In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Valor
0,vcs acham msm que a câmera d xiaomi é melhor q...,1
1,pq as autorizadas do xiaomi só são no toba do ...,1
2,🔥 em stock! smart tvs xiaomi ao preço que quer...,0
3,xiaomi poco x3 é um verdadeiro sucesso de vend...,1
4,quero refazer pq na época eu usava o iphone 7 ...,1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Escolhemos a marca Xiaomi e consideramos, para fins de análise, que os tweets relevantes são aqueles que elogiam de alguma forma, pedem melhorias ou criticam a marca e seus produtos. Na categoria não relevantes foram classificados os tweets de propagandas de lojas e aqueles que mencionam o nome da marca mas não se encaixam nas categorias já mencionadas.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [5]:
# https://docs.python.org/3/library/re.html#
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [6]:
# Limpando a base de dados para Treinamento
treinamento_limpo = cleanup(" ".join(train.Treinamento).lower())
palavras_train = treinamento_limpo.split()

In [7]:
serie_train = pd.Series(palavras_train)
serie_train

0                    @miuirom
1                        miui
2                          12
3                          no
4                         rn8
                ...          
7171                      g5s
7172                   xiaomi
7173                       mi
7174                       a2
7175    https//tco/gtqmqfuxso
Length: 7176, dtype: object

In [8]:
tabela_train_absoluta = serie_train.value_counts()
tabela_train_absoluta.head()

xiaomi    361
o         190
e         156
de        156
que       155
dtype: int64

In [9]:
tabela_train_relativa = serie_train.value_counts(True)
tabela_train_relativa.head()

xiaomi    0.050307
o         0.026477
e         0.021739
de        0.021739
que       0.021600
dtype: float64

In [10]:
# Separação de twittes em relevantes e irrelevantes
dados_R = train.loc[train.Valor == 1,:]
dados_I = train.loc[train.Valor == 0,:]

In [11]:
### Organizando a base de dados separada em relevantes e irrelevantes
dados_R_limpo = cleanup(" ".join(dados_R.Treinamento).lower())
dados_I_limpo = cleanup(" ".join(dados_I.Treinamento).lower())

In [12]:
# Separação de palavras pertencentes a twittes relevantes e irrelevantes
palavras_dados_R = dados_R_limpo.split()
palavras_dados_I = dados_I_limpo.split()

In [13]:
# Probabilidade de um twitte ser relevante ou irrelevante
prob_relev = train.Valor.value_counts(True, sort=False)
P_R = prob_relev[1]
P_I = prob_relev[0]
print(P_R)
print(P_I)

0.41456582633053224
0.5854341736694678


In [14]:
tabela_R = pd.Series(palavras_dados_R).value_counts()
tabela_R.head()

xiaomi    152
o         103
a          84
redmi      84
e          84
dtype: int64

In [15]:
tabela_I = pd.Series(palavras_dados_I).value_counts()
tabela_I.head()

xiaomi    209
o          87
que        76
de         74
e          72
dtype: int64

In [16]:
total_R = tabela_R.sum()
total_R

3572

In [17]:
total_I = tabela_I.sum()
total_I

3604

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [18]:
frase = "quero refazer pq na época eu usava o iphone 7 e agora eu tenho um xiaomi top e vai ficar bem mais legal"

In [19]:
frase = cleanup(frase.lower())

In [20]:
frase = frase.split()
frase

['quero',
 'refazer',
 'pq',
 'na',
 'época',
 'eu',
 'usava',
 'o',
 'iphone',
 '7',
 'e',
 'agora',
 'eu',
 'tenho',
 'um',
 'xiaomi',
 'top',
 'e',
 'vai',
 'ficar',
 'bem',
 'mais',
 'legal']

In [21]:
Prob_frase_R = (tabela_R.reindex(frase,fill_value = 0)+1)/(total_R + len(frase)).prod()
Prob_frase_R*P_R

quero      0.000461
refazer    0.000115
pq         0.001153
na         0.001960
época      0.000231
eu         0.005535
usava      0.000115
o          0.011993
iphone     0.001384
7          0.001268
e          0.009802
agora      0.000807
eu         0.005535
tenho      0.001268
um         0.006458
xiaomi     0.017644
top        0.000346
e          0.009802
vai        0.000461
ficar      0.000346
bem        0.000692
mais       0.002537
legal      0.000115
dtype: float64

In [37]:
# Limpando e criando uma coluna com cada tweet
cont = 0
for a in test.Teste:
    teste_limpo = cleanup(a).lower()
    test.loc[cont,'Limpos'] = teste_limpo
    cont += 1

In [38]:
test.head()

,Teste,Valor,Limpos
0,vcs acham msm que a câmera d xiaomi é melhor q...,1,vcs acham msm que a câmera d xiaomi é melhor q...
1,pq as autorizadas do xiaomi só são no toba do ...,1,pq as autorizadas do xiaomi só são no toba do ...
2,🔥 em stock! smart tvs xiaomi ao preço que quer...,0,🔥 em stock smart tvs xiaomi ao preço que queri...
3,xiaomi poco x3 é um verdadeiro sucesso de vend...,1,xiaomi poco x3 é um verdadeiro sucesso de vend...
4,quero refazer pq na época eu usava o iphone 7 ...,1,quero refazer pq na época eu usava o iphone 7 ...


In [39]:
type(test.Teste)

pandas.core.series.Series

In [84]:
# Calculando a probabilidade de cada tweet ser relevante ou irrelevante
lista_palavras_separadas = []
for frase in test.Limpos:
    lista_palavras_separadas.append(frase.split())
print(lista_palavras_separadas[2])

['🔥', 'em', 'stock', 'smart', 'tvs', 'xiaomi', 'ao', 'preço', 'que', 'querias', '👉', 'https//tco/dadrkzme3p', 'xiaomi', 'tv', 'smarttv', 'televisor', 'https//tco/wlvxfqphrj']


In [86]:
cont2 = 0
for p in lista_palavras_separadas:
    Prob_frase_R = (tabela_R.reindex(p,fill_value = 0)+1)/(total_R + len(p)).prod()
    Prob_frase_I = (tabela_I.reindex(p,fill_value = 0)+1)/(total_I + len(p)).prod()
    if (Prob_frase_R*P_R) > (Prob_frase_I*P_I):
        test.loc[cont2,'Classificador'] = 1
    else:
        test.loc[cont2,'Classificador'] = 0
    cont2 += 1

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
"""
i = 0
a = 0
while i <= len(lista_palavras_separadas) and a <= len(lista_palavras_separadas[i]):
    Prob_frase_R = (tabela_R.reindex(lista_palavras_separadas[i][a],fill_value = 0)+1)/(total_R + len(lista_palavras_separadas[i])).prod()
    Prob_frase_I = (tabela_I.reindex(lista_palavras_separadas[i][a],fill_value = 0)+1)/(total_I + len(lista_palavras_separadas[i])).prod()
    if (Prob_frase_R*P_R) > (Prob_frase_I*P_I):
        test.loc[i,'Classificador'] = 1
    else:
        test.loc[i,'Classificador'] = 0
    i += 1
    a += 1
    """

In [25]:
# Tabela comparativa da classificação do modelo com a nossa
tab = pd.crosstab(test['Classificador'],test['Valor'], normalize = True)

KeyError: 'Classificador'

In [26]:
tab[0][0]

NameError: name 'tab' is not defined

In [27]:
# Parâmetros obtidos a partir da tabela
Porc_R_classif_R = ###
Porc_I_classif_R = 
Porc_I_classif_I = 
Porc_R_classif_I = 

print(f'Porcentagem de verdadeiros positivos = {Porc_R_classif_R}'
print(f'Porcentagem de falsos positivos = {Porc_I_classif_R}'
print(f'Porcentagem de verdadeiros negativos = {Porc_I_classif_I}'  
print(f'Porcentagem de falsos negativos = {Porc_R_classif_I}'

SyntaxError: invalid syntax (<ipython-input-27-2649fea1352a>, line 2)

In [28]:
"""
# Conta quantas vezes a palavra apareceu na base de dados
c = 0 
for p in palavras_train:
    if p == frase[i]:
        c += 1
# Calcula a probabilidade de uma palavra dada uma frase e a base de dados com todas as palavras
c = 0 
P = 1
for i in len(frase):
    for p in palavras_train:
        if p == frase[i]:
            c += 1
    P *= ((c+1/(len(frase)+len(set(palavras_train)))))

 
   
    probFraseR = 1 *tabela_train_relativa[frase[i]]
    probFraseI = 1 *tabela_train_relativa[frase[i]]
    if frase[i] not in tabela_train_relativa:
        frase[i] = 1
        """

'\n# Conta quantas vezes a palavra apareceu na base de dados\nc = 0 \nfor p in palavras_train:\n    if p == frase[i]:\n        c += 1\n# Calcula a probabilidade de uma palavra dada uma frase e a base de dados com todas as palavras\nc = 0 \nP = 1\nfor i in len(frase):\n    for p in palavras_train:\n        if p == frase[i]:\n            c += 1\n    P *= ((c+1/(len(frase)+len(set(palavras_train)))))\n\n \n   \n    probFraseR = 1 *tabela_train_relativa[frase[i]]\n    probFraseI = 1 *tabela_train_relativa[frase[i]]\n    if frase[i] not in tabela_train_relativa:\n        frase[i] = 1\n        '

___
### Concluindo

In [ ]:
### Conferir emojis - espaços? não deleta mesmo?

Faça um comparativo qualitativo sobre os percentuais obtidos para que possa discutir a
performance do seu classificador.

###Explique como são tratadas as mensagens com dupla negação e sarcasmo.

###Proponha um plano de expansão. Por que eles devem continuar financiando o seu
projeto? - com as melhoris sugeridas abaixo?

Discorrer por que não posso alimentar minha base de Treinamento
automaticamente usando o próprio classificador, aplicado a novos tweets.

     ### A base de treinamento não pode ser alimentada automaticamente com o uso do próprio classificador uma vez que este aprende com os tweets já classificados, manualmente, como relevantes ou irrelevantes. Dessa forma, se o classificador criasse novos tweets para sua base de treinamento, ele não estaria aperfeiçoando a sua análise, mas sim aplicando aquilo que já aprendeu anteriomente, então não haveria melhora na precisão das previsões.

Propor diferentes cenários de uso para o classificador Naive-Bayes. Pense em
outros cenários sem intersecção com este projeto.

    Dentre os cenários em que pode-se usar o classificador Naive-Bayes, têm-se:
    - Filtrar anúncios em sites de forma personalizada para cada usuário, classificando como relevantes aqueles que anunciam produtos, sites e serviços que a pessoa usualmente busca;
    - Filtrar e-mails, classificando-os como spam, principais, etc. de acordo com o conteúdo (Por exemplo: se é propaganda ou não), com o remetente (Por exemplo: se é um contato frequente ou uma conta comercial), dentre outras variàveis;
    ###- Configurar um site com filtros para mostrar produtos a partir de caraterísticas selecionadas pelo usuário (Por exemplo: Se um usuário busca pela cor rosa, produtos com essa caractrística serão dados como relevantes e aparecerão como resultado dessa busca).

Sugerir e explicar melhorias reais no classificador com indicações concretas de
como implementar (não é preciso codificar, mas indicar como fazer. Indique
material de pesquisa sobre o assunto).

    Sugestões para melhoria na performance do nosso classificador:
    - Aumentar a base de dados considerada para a construção do classificador (aumentando o valor do parâmetro "n" no   arquivo jupyter "Projeto1_Obtenção_dos_tweets.ipynb", que define a quantia mínima de mensagens capturadas para a base de treinamento e classificando-os manualmente como relevantes ou irrelevantes). A partir dessa modificação, o classificador terá mais dados nos quais se basear para fundamentar a classificação de futuros tweets, reduzindo a taxa de erros nesse processo;
    - Criar categorias intermediárias de relevância considerando a probabilidade (Para tal, deve-se criar novas variáveis que classificam cada faixa de valor, dada em probabilidade, em uma categoria. Ademais, deve-se adicionar essa classificação a uma coluna nova do tipo categórica no Dataframe). Considerando essa adaptação, a probabilidade de uma frase relevante, mesmo que em grau pequeno, ser classificada como irrelevante será reduzida.
    - ########

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis  OK
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis #######
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação #########
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento OK
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto OK
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa) OK

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

In [ ]:
"""
mt_relev = (P_R >= )|( == )
relev = ( == )|( == )
neutro = (dados['SENTIMENTO'] == 'Indiferente')
irr = (dados['SENTIMENTO'] == 'Indiferente')
mt_irr = (dados['SENTIMENTO'] == 'Indiferente')

dados.loc[mt_relev,'SENTIMENTO'] = 'Muito Relevante'
dados.loc[relev,'SENTIMENTO'] = 'Relevante'
dados.loc[neutro,'SENTIMENTO'] = 'Neutro'
dados.loc[irr,'SENTIMENTO'] = 'Irrelevante'
dados.loc[mt_irr,'SENTIMENTO'] = 'Muito Irrelevante'

dados['SENTIMENTO'] = dados['SENTIMENTO'].astype('category')
dados['SENTIMENTO'] = pd.Categorical(dados.SENTIMENTO, 
                                     categories = ['Muito Relevante', 'Relevante', 'Neutro', 'Irrelevante', 'Muito Irrelevante'], 
                                     ordered=True)
dados['SENTIMENTO'] = dados['SENTIMENTO'].cat.as_ordered()
dados['SENTIMENTO']

"""